In [ ]:
! pip install -U sentence-transformers
! pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import math
import json
from datasets import load_dataset, DatasetDict, Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, util, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import numpy as np

2024-06-12 07:02:25.254100: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 07:02:25.254244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 07:02:25.388349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
no_segment_flatten_corpus = []
with open("/kaggle/input/law-qa/flatten_law_corpus.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_flatten_corpus.append(data)

In [4]:
no_segment_queries = []

with open("/kaggle/input/law-qa/query_set_evaluate.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_queries.append(data)

In [5]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(no_segment_queries, test_size=0.2, random_state=42)

In [6]:
no_segment_flatten_queries_train = []
for query in train_set:
    for rd in query['relevant_docs']:
        no_segment_flatten_queries_train.append({
            'query_id': query['query_id'],
            'query': query['query'],
            'relevant_doc': rd,
        })

In [7]:
def process_relevant_doc_id(relevant_doc_id):
    second_underscore_index = relevant_doc_id.find("_", relevant_doc_id.find("_") + 1)
    if second_underscore_index != -1:
        id_part = relevant_doc_id[:second_underscore_index]
        chunk_part = relevant_doc_id[second_underscore_index + 1:]
        chunk_part = int(chunk_part) if chunk_part.isdigit() else None
        return id_part, chunk_part
    else:
        return relevant_doc_id, None

In [8]:
import re
def chunk_text_with_subheadings(text):
    if re.search(r'\d+\.', text) or re.search(r'[a-z]\)', text, flags=re.IGNORECASE):
        chunks = re.split(r'\n(?=\d+\.)', text)
        final_chunks = [chunk.strip() for chunk in chunks]
    else:
        final_chunks = [text.strip()]
    
    return final_chunks

In [9]:
def get_first_chunk_with_part(list_chunks, chunk_part):
    pattern = re.compile(rf'^\s*{chunk_part}\s*[\.\)\s]')
    
    for chunk in list_chunks:
        if pattern.match(chunk):
            return chunk
    
    return None

In [10]:
no_segment_data_train = []
corpus_dict = {doc['article_id']: doc for doc in no_segment_flatten_corpus}

for query in no_segment_flatten_queries_train:
    relevant_doc_id, chunk_part = process_relevant_doc_id(query['relevant_doc'])
    
    if relevant_doc_id in corpus_dict:
        matched_doc = corpus_dict[relevant_doc_id]
        if chunk_part is None:
            content_str = ' '.join(map(str, matched_doc['content']))
            positive = f"{matched_doc['name']}. {content_str}"
            no_segment_data_train.append({
                'query': query['query'],
                'positive': positive
            })
        else:
            list_chunks = chunk_text_with_subheadings(' '.join(map(str, matched_doc['content'])))
            chunk_part_match = get_first_chunk_with_part(list_chunks, chunk_part )
            if chunk_part_match is not None:
                no_segment_data_train.append({
                'query': query['query'],
                'positive': f"{matched_doc['name']}. {chunk_part_match}"
            })

In [11]:
segmented_data_train = []
for query in no_segment_data_train:
    segmented_data_train.append({
        'query' : ' '.join(rdrsegmenter.word_segment(query['query'])),
        'positive' : ' '.join(rdrsegmenter.word_segment(query['positive']))
    })

In [12]:
train_dataset = []
train_dataset = [InputExample(texts = [a['query'], a['positive']]) for a in segmented_data_train]

In [13]:
word_embedding_model = models.Transformer("bkai-foundation-models/vietnamese-bi-encoder", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [14]:
num_epochs = 5
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)

In [15]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_class=torch.optim.AdamW,
    optimizer_params={
        'lr': 2e-5
    },
    weight_decay=0.01,
    output_path="/kaggle/working/finetuned_bkai_5epoch_15k"
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.089000
1000,0.021900
1500,0.014900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# Evalute

In [16]:
import json
import pickle

with open('/kaggle/input/law-train-evalute/chunk_id_mapping', 'rb') as f:
    idx_mapping = pickle.load(f)
with open('/kaggle/input/law-train-evalute/chunk_corpus', 'rb') as f:
    segmented_chunk_corpus = pickle.load(f)
with open('/kaggle/input/law-train-evalute/queries_relevant_chunks', 'rb') as f:
    segmented_queries_relevant_chunk = pickle.load(f)
    
query_test_set = test_set

In [17]:
query_test_relevant_chunks = []

for query in query_test_set:
    
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        try:
            relevant_chunks += idx_mapping[relevant_doc]
        except KeyError:
            continue
        
    query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : query['query'],
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : relevant_chunks
    })

In [18]:
segmented_query_test_relevant_chunks = []

for query in query_test_relevant_chunks:
    segmented_query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : ''.join(rdrsegmenter.word_segment(query['query'])),
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : query['relevant_chunks']
    })

In [19]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = {}
queries = {}
relevant_docs = {}
corpus_idx = 0
query_idx = 0

for query in segmented_query_test_relevant_chunks:
    queries[query['query_id']] = query['query']
    relevant_docs[query['query_id']] = set(query['relevant_chunks'])

for doc in segmented_chunk_corpus:
    corpus[doc['chunk_id']] = doc['text']

## Evalute 15k - 5 epochs

In [20]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_bkai_5epoch_15k_chunk")

{'cosine_accuracy@1': 0.5648414985590778,
 'cosine_accuracy@3': 0.7748044462741869,
 'cosine_accuracy@5': 0.8291477974475092,
 'cosine_accuracy@10': 0.8855496088925484,
 'cosine_precision@1': 0.5648414985590778,
 'cosine_precision@3': 0.44860710854947167,
 'cosine_precision@5': 0.3718402634829148,
 'cosine_precision@10': 0.24310415808974886,
 'cosine_recall@1': 0.19225235775462207,
 'cosine_recall@3': 0.4311430008702549,
 'cosine_recall@5': 0.5636388789229464,
 'cosine_recall@10': 0.7019417663973037,
 'cosine_ndcg@10': 0.6130117774823358,
 'cosine_mrr@10': 0.6801059616930345,
 'cosine_map@100': 0.5461956164842632,
 'dot_accuracy@1': 0.5516673528200906,
 'dot_accuracy@3': 0.7612186084808563,
 'dot_accuracy@5': 0.8242074927953891,
 'dot_accuracy@10': 0.8843145327295183,
 'dot_precision@1': 0.5516673528200906,
 'dot_precision@3': 0.44243172773432143,
 'dot_precision@5': 0.3659942363112392,
 'dot_precision@10': 0.2410868670234664,
 'dot_recall@1': 0.18617577115106962,
 'dot_recall@3': 0.42

In [21]:
import pandas as pd
df = pd.read_csv('/kaggle/working/finetuned_bkai_5epoch_15k_chunk/Information-Retrieval_evaluation_results.csv')
df

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.564841,0.774804,0.829148,0.88555,0.564841,0.192252,0.448607,0.431143,...,0.186176,0.442432,0.422995,0.365994,0.554637,0.241087,0.696715,0.669249,0.603661,0.534786
